In [ ]:
#imports

In [ ]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, Dense, RepeatVector, Subtract, Add, TimeDistributed, UpSampling1D, Embedding, Flatten, BatchNormalization, Activation, Lambda, Conv1D, MaxPooling1D
from keras.losses import mse, binary_crossentropy, categorical_crossentropy
from keras.utils import plot_model
from keras.optimizers import SGD, adam
from tensorflow.keras.models import load_model
from keras import backend as K
from keras.engine import Layer, InputSpec
from keras.initializers import Constant

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from keras.wrappers.scikit_learn import KerasRegressor

from read_nilmtk import read_data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.ndimage.filters as filters

In [ ]:
#utils

In [ ]:
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.
    # Arguments
        args (tensor): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def class_preprocess(latent_test, latent_ran_test):
	L = []
	len(L)
	for x in latent_test:
		L.append([x,0])
	for x in latent_ran_test:
		L.append([x,1])

	L = np.array(L)
	np.random.shuffle(L)
	L.shape
	L[0]
	L[1]

	inputs = [x[0] for x in L]
	targets = [x[1] for x in L]

	return inputs, targets


In [ ]:
#load data

In [ ]:
x_train, x_test, x_train_shifted_8, x_test_shifted_8, v_train, v_test  = read_data(1,'kettle',1)
x_train = np.squeeze(x_train, axis=2)
x_test = np.squeeze(x_test, axis=2)
x_train_shifted_8 = np.squeeze(x_train_shifted_8, axis=2)
x_test_shifted_8 = np.squeeze(x_test_shifted_8, axis=2)
v_train = np.squeeze(v_train, axis=2)
v_test = np.squeeze(v_test, axis=2)

#train_set = np.concatenate((x_train,x_train_shifted_8), axis=0)
train_set = x_train
test_set = x_test
pos_shifted_train_set = x_train_shifted_8
pos_shifted_set = x_test_shifted_8
neg_shifted_set = v_test

features_matrix_train, labels_train = class_preprocess(x_train, x_train_shifted_8)
features_matrix_train = np.array(features_matrix_train)
labels_train = np.array(labels_train)
features_matrix_test, labels_test = class_preprocess(x_test, x_test_shifted_8)
features_matrix_test = np.array(features_matrix_test)
labels_test = np.array(labels_test)

In [ ]:
#define VAE

In [ ]:
def var_shallow(optimizer='Nadam', mid_activation='linear', out_activation='sigmoid', kl_coef=1):
    
    def my_vae_loss(y_true, y_pred):
        reconstruction_loss = 24*categorical_crossentropy(y_true, y_pred)
        kl_loss = - 0.5 * K.sum(1 + z_log_var - 0.001*K.square(z_mean) - K.exp(z_log_var), axis=-1)

        vae_loss = K.mean(reconstruction_loss + kl_coef*kl_loss)
        return vae_loss


    input_window = Input(shape=(window_length,))

    z_mean = Dense(latent_size, name='z_mean', activation=mid_activation)(input_window)
    z_log_var = Dense(latent_size, name='z_log_var', activation=mid_activation)(input_window)
    
    z = Lambda(sampling, output_shape=(latent_size,), name='z')([z_mean, z_log_var])
    z = Activation('sigmoid')(z)

    decoded = Dense(window_length, activation=out_activation)(z)

    encoder = Model(input_window, [z_mean, z_log_var, z])
    autoencoder = Model(input_window, decoded)

    autoencoder.compile(optimizer=optimizer, loss=my_vae_loss)
    
    return autoencoder, encoder


In [ ]:
#hyperparameters

In [ ]:
latent_size = 2
window_length=24
optimizer='Nadam'
mid_activation='sigmoid'
out_activation='sigmoid'

In [ ]:
#create and train the AE

In [ ]:
autoencoder, encoder = var_shallow()
history = autoencoder.fit(train_set, train_set,
                epochs=200,
                batch_size=32, shuffle=True)

In [ ]:
#get the mean and var latent spaces of the VAE

In [ ]:
latent_train = encoder.predict(train_set)
latent_shifted_train = encoder.predict(pos_shifted_train_set)
latent_test = encoder.predict(test_set)
latent_shifted_pos = encoder.predict(pos_shifted_set)

#extract means
mean_train = latent_train[0]
mean_shifted_train = latent_shifted_train[0]
mean_test = latent_test[0]
mean_shifted_pos = latent_shifted_pos[0]

#extract vars
var_train = latent_train[1]
var_shifted_train = latent_shifted_train[1]
var_test = latent_test[1]
var_shifted_pos = latent_shifted_pos[1]

In [ ]:
#rearange it into a 4-dim vectors to be fed to the classifier

x,y = mean_train.T
w,z = var_train.T
c = zip(x,y,w,z)
latent_train = np.array(c)

x,y = mean_shifted_train.T
w,z = var_shifted_train.T
c = zip(x,y,w,z)
latent_shifted_train = np.array(c)

x,y = mean_test.T
w,z = var_test.T
c = zip(x,y,w,z)
latent_test = np.array(c)

x,y = mean_shifted_pos.T
w,z = var_shifted_pos.T
c = zip(x,y,w,z)
latent_shifted_pos = np.array(c)

In [ ]:
#visualize the latent spaces

In [ ]:
if latent_size == 2:
    X,Y,W,Z = latent_test.T
    XN,YN,WN,ZN = latent_shifted_pos.T
    #XNN,YNN = latent_shifted_neg.T
    plt.subplot(1, 2, 1)
    plt.plot(X, Y, 'go')
    plt.plot(XN, YN, 'ro')
    plt.title('Means')
    plt.subplot(1, 2, 2)
    plt.plot(W, Z, 'go')
    plt.plot(WN, ZN, 'ro')
    plt.title('Variability')
    #plt.plot(XNN, YNN, 'yo')
elif latent_size ==3:
    from mpl_toolkits.mplot3d import axes3d

    data = (latent_test, latent_shifted_pos,latent_shifted_neg)
    colors = ("green", "red", "yellow")
    groups = ("normal", "anomaly", "anomaly")

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax = fig.gca(projection='3d')


    #X,Y,Z = latent_test.T
    #XN,YN,ZN = latent_shifted_pos.T
    for data, color, group in zip(data, colors, groups):
        x, y, z = data.T
        ax.scatter(x, y, z, alpha=0.8, c=color, edgecolors='none', s=30, label=group)
else:
    pass

In [ ]:
#prepare latent data for classification
#train
features_matrix_train, labels_train = class_preprocess(latent_train, latent_shifted_train)
#test
features_matrix_test, labels_test = class_preprocess(latent_test, latent_shifted_pos)

In [ ]:
#create and train the classifier
classifier = RandomForestClassifier()
classifier.fit(features_matrix_train, labels_train)

In [ ]:
#predict and evaluate
predicted_labels_test = classifier.predict(np.array(features_matrix_test))
print(confusion_matrix(labels_test,predicted_labels_test))
print(classification_report(labels_test,predicted_labels_test))